### TO DO:

- correct the path for the pipeline
- correct the pathes


In [18]:
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage import data, exposure
from PIL import Image
from sklearn.svm import SVC #SVM classifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray

In [19]:
def normalize_and_flatten(features):
    features = features - np.min(features, axis=0, keepdims=True)
    features = features / np.max(features, axis=0, keepdims=True)
    return features.flatten()


In [20]:
resize_factor = (188, 250)

train_img_path = '../data/train_set/train_images/'
test_img_path = '../data/train_set/test_images/'
train_features_path = '../data/train_set/train_points/'
test_features_path = '../data/train_set/test_points/'
labels = pd.read_csv('../data/train_set/label.csv')

train_img = sorted(os.listdir(train_img_path))
test_img = sorted(os.listdir(test_img_path))

## TRUE Labels

In [4]:
### Train emotion indexes
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0]

### Test emotion indexes
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)
    
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0]   

## FEATURES
#### HOGS, train_features, concat_train_features

In [5]:
### extract HOG for train data set

HOGs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time))  

### extract HOG for test data set

HOGs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs_test.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time)) 


--- 124.47650098800659 seconds ---
--- 30.952392101287842 seconds ---


In [21]:
### Point features for train dataset (NORMALIZED VERSION)
train_features = sorted(os.listdir(train_features_path))
train_features_list = []
for f in train_features:
    tmp = sio.loadmat(os.path.join(train_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    train_features_list.append(normalize_and_flatten(features))
    
### Point features for test dataset (NORMALIZED VERSION)
test_features = sorted(os.listdir(test_features_path))
test_features_list = []
for f in test_features:
    tmp = sio.loadmat(os.path.join(test_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    test_features_list.append(normalize_and_flatten(features))    

In [24]:
test_features_list[1].shape

(156,)

In [39]:
### Cancate of HOG and .mat features train
start_time = time.time()
pca = PCA(n_components=100)
reduced_hogs = pca.fit_transform(np.array(HOGs)) 
train_np = np.array(train_features_list)
concat_train_features = np.concatenate((reduced_hogs, train_np), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

### test
start_time = time.time()
reduced_hogs_test = pca.transform(np.array(HOGs_test))
test_np = np.array(test_features_list)
concat_test_features = np.concatenate((reduced_hogs_test, test_np), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'HOGs' is not defined

In [5]:
### extract LBP for train data set

METHOD = 'uniform'
radius = 3
n_points = 8 * radius

LBPs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    img = rgb2gray(img)
#     print(img.shape)    
    fetures_lbp = local_binary_pattern(img, n_points, radius, METHOD)
    LBPs.append(fetures_lbp.flatten())

print("--- %s seconds ---" % (time.time() - start_time))  

### extract LBP for test data set

LBPs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    img = rgb2gray(img)
    fetures_lbp = local_binary_pattern(img, n_points, radius, METHOD)
    LBPs_test.append(fetures_lbp.flatten())
print("--- %s seconds ---" % (time.time() - start_time)) 


--- 103.11895799636841 seconds ---
--- 26.66599416732788 seconds ---


## CLASSIFIERS

#### Linear SVM on HOG

In [9]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(np.array(HOGs), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(HOGs), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(HOGs_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 482.69038796424866 seconds
Train Accuracy Durtion: 234.04115295410156 seconds
Train Accuracy: 0.09404702351175588
Test Accuracy Duration: 60.57271194458008 seconds
Test Accuracy: 0.06786427145708583


#### Linear SVM on LBP

In [10]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(np.array(LBPs), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(LBPs), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(LBPs_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 473.7649929523468 seconds
Train Accuracy Durtion: 236.37778902053833 seconds
Train Accuracy: 0.9994997498749375
Test Accuracy Duration: 56.19797611236572 seconds
Test Accuracy: 0.02594810379241517


#### Linear SVM on PCA on LBP

In [11]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
linear_SVM = SVC(kernel="linear", C=0.0001)
linear_SVM.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = linear_SVM.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 12.468503713607788 seconds
Train Accuracy Durtion: 0.9384250640869141 seconds
Train Accuracy: 0.9994997498749375
Test Accuracy Duration: 0.6064751148223877 seconds
Test Accuracy: 0.041916167664670656


#### Linear SVM on .mat features

In [64]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=100)
linear_SVM.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.6397969722747803 seconds
Train Accuracy Durtion: 0.7331209182739258 seconds
Train Accuracy: 0.7928964482241121
Test Accuracy Duration: 0.17191791534423828 seconds
Test Accuracy: 0.4630738522954092


#### L SVM on concat features

In [67]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.5)
linear_SVM.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 2.706681966781616 seconds
Train Accuracy Durtion: 1.2310597896575928 seconds
Train Accuracy: 0.7898949474737369
Test Accuracy Duration: 0.32773709297180176 seconds
Test Accuracy: 0.11377245508982035


#### L SVM on PCA HOG

In [ ]:
start_time = time.time()
# pca = PCA(n_components=200)
# reduced_features = pca.fit_transform(np.array(HOGs))
linear_SVM = SVC(kernel="linear", C=1)
linear_SVM.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = linear_SVM.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### SVM on PCA HOG

In [38]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = SVC(gamma='auto', C=10)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 15.348134279251099 seconds
Train Accuracy Durtion: 0.5401968955993652 seconds
Train Accuracy: 0.9934967483741871
Test Accuracy Duration: 0.5810391902923584 seconds
Test Accuracy: 0.04590818363273453


In [16]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = SVC(gamma='auto', C=0.01)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 11.554597854614258 seconds
Train Accuracy Durtion: 0.9510560035705566 seconds
Train Accuracy: 0.05202601300650325
Test Accuracy Duration: 0.666874885559082 seconds
Test Accuracy: 0.06187624750499002


#### SVM on .mat Features

In [30]:
start_time = time.time()
model = SVC(gamma='auto', C=1000)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.7238380908966064 seconds
Train Accuracy Durtion: 0.8284599781036377 seconds
Train Accuracy: 0.6183091545772886
Test Accuracy Duration: 0.1888411045074463 seconds
Test Accuracy: 0.46506986027944114


#### SVM on concat features

In [37]:
start_time = time.time()
model = SVC(gamma='auto', C=10)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 2.657891035079956 seconds
Train Accuracy Durtion: 1.3531501293182373 seconds
Train Accuracy: 0.8539269634817409
Test Accuracy Duration: 0.36635494232177734 seconds
Test Accuracy: 0.07584830339321358


#### RandomForestClassifier on PCA HOG

In [42]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = RandomForestClassifier(max_depth=5, n_estimators=50)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 11.663228273391724 seconds
Train Accuracy Durtion: 0.02125692367553711 seconds
Train Accuracy: 0.5317658829414708
Test Accuracy Duration: 0.337507963180542 seconds
Test Accuracy: 0.06187624750499002


In [27]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 16.7584331035614 seconds
Train Accuracy Durtion: 0.03023386001586914 seconds
Train Accuracy: 0.6543271635817909
Test Accuracy Duration: 0.48764681816101074 seconds
Test Accuracy: 0.05788423153692615


#### RFC on .mat features

In [47]:
start_time = time.time()
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 1.1041409969329834 seconds
Train Accuracy Durtion: 0.030183076858520508 seconds
Train Accuracy: 0.5157578789394698
Test Accuracy Duration: 0.011514902114868164 seconds
Test Accuracy: 0.3013972055888224


#### RFC on cancate features

In [48]:
start_time = time.time()
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 1.3742477893829346 seconds
Train Accuracy Durtion: 0.05086398124694824 seconds
Train Accuracy: 0.5327663831915957
Test Accuracy Duration: 0.025084972381591797 seconds
Test Accuracy: 0.3073852295409182


#### MLPClassifier on pca HOG

In [49]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 19.290612936019897 seconds
Train Accuracy Durtion: 0.04608917236328125 seconds
Train Accuracy: 0.9904952476238119
Test Accuracy Duration: 0.9957513809204102 seconds
Test Accuracy: 0.08183632734530938


In [26]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 19.16475510597229 seconds
Train Accuracy Durtion: 0.056764841079711914 seconds
Train Accuracy: 0.9984992496248124
Test Accuracy Duration: 0.4417610168457031 seconds
Test Accuracy: 0.05189620758483034


#### MLP on .mat

In [50]:
start_time = time.time()
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 3.587941884994507 seconds
Train Accuracy Durtion: 0.0499267578125 seconds
Train Accuracy: 0.1265632816408204
Test Accuracy Duration: 0.011600971221923828 seconds
Test Accuracy: 0.10778443113772455


#### MLP on concat features

In [52]:
start_time = time.time()
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 60, 40))
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 5.981200933456421 seconds
Train Accuracy Durtion: 0.04111790657043457 seconds
Train Accuracy: 0.9959979989994997
Test Accuracy Duration: 0.010310888290405273 seconds
Test Accuracy: 0.11976047904191617


#### KNN on .mat features

In [55]:
start_time = time.time()
model = KNeighborsClassifier(n_neighbors=5)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.008569002151489258 seconds
Train Accuracy Durtion: 1.2204737663269043 seconds
Train Accuracy: 0.4497248624312156
Test Accuracy Duration: 0.288834810256958 seconds
Test Accuracy: 0.22355289421157684


#### KNN on pca HOG

In [60]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = KNeighborsClassifier(n_neighbors=10)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 14.751265048980713 seconds
Train Accuracy Durtion: 1.218137264251709 seconds
Train Accuracy: 0.1905952976488244
Test Accuracy Duration: 0.602686882019043 seconds
Test Accuracy: 0.029940119760479042


#### KNN on concate features

In [59]:
start_time = time.time()
model = KNeighborsClassifier(n_neighbors=3)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.02538895606994629 seconds
Train Accuracy Durtion: 1.2404861450195312 seconds
Train Accuracy: 0.3476738369184592
Test Accuracy Duration: 0.32026219367980957 seconds
Test Accuracy: 0.015968063872255488


In [74]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

start_time = time.time()
model = AdaBoostClassifier(n_estimators=150)
scores = cross_val_score(model, np.array(train_features_list), emotion_idx, cv=5)
print("Train Duration : %s seconds" % (time.time() - start_time)) 
print(scores)

# model.fit(np.array(train_features_list), emotion_idx)
# print("Train Duration : %s seconds" % (time.time() - start_time)) 

# start_time = time.time()
# accuracy = model.score(np.array(train_features_list), emotion_idx)
# print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
# print("Train Accuracy: {}".format(accuracy))

# start_time = time.time()
# accuracy = model.score(np.array(test_features_list), test_emotion_idx)
# print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
# print("Test Accuracy: {}".format(accurac

Train Duration : 23.674659967422485 seconds
[0.18382353 0.15880893 0.12219451 0.13636364 0.10741688]


In [78]:
lbp.shape

(1999, 156)

In [10]:
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb

# settings for LBP
radius = 3
n_points = 8 * radius


def overlay_labels(image, lbp, labels):
    mask = np.logical_or.reduce([lbp == each for each in labels])
    return label2rgb(mask, image=image, bg_label=0, alpha=0.5)


def highlight_bars(bars, indexes):
    for i in indexes:
        bars[i].set_facecolor('r')


image = data.brick()

In [12]:
image.shape

(512, 512)

In [36]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

start_time = time.time()
model = LDA()
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.05864381790161133 seconds
Train Accuracy Durtion: 0.004312276840209961 seconds
Train Accuracy: 0.7563781890945472
Test Accuracy Duration: 0.0032968521118164062 seconds
Test Accuracy: 0.5508982035928144


/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


In [37]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = LDA()
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Train Duration : 15.780615091323853 seconds
Train Accuracy Durtion: 0.003587007522583008 seconds
Train Accuracy: 0.3871935967983992
Test Accuracy Duration: 0.7522678375244141 seconds
Test Accuracy: 0.03592814371257485


In [35]:
import sklearn
sklearn.__version__

'0.19.1'

In [42]:
### calculating the distance
def normalize(features):
    features = features - np.min(features, axis=0, keepdims=True)
    features = features / np.max(features, axis=0, keepdims=True)
    return features

train_features = sorted(os.listdir(train_features_path))
train_features_list2 = []
for f in train_features:
    tmp = sio.loadmat(os.path.join(train_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    train_features_list2.append(normalize(features))
    
test_features = sorted(os.listdir(test_features_path))
test_features_list2 = []
for f in test_features:
    tmp = sio.loadmat(os.path.join(test_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    test_features_list2.append(normalize(features)) 

In [43]:
train_features_list2[0].shape

(78, 2)

In [47]:
import math
def euclideanـdistance(p1, p2):
    tmp = ((p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1]))
    return math.sqrt(tmp)

dist = []
for i, p1 in enumerate(train_features_list2[0]):
    for j, p2 in enumerate(train_features_list2[0]):
        if j > i:
            dist.append(euclideanـdistance(p1, p2))

In [53]:
euclideanـistance_train = []
euclideanـdistance_test = []

start_time = time.time()
for im in train_features_list2:
    dist = []
    for i, p1 in enumerate(im):
        for j, p2 in enumerate(im):
            if j > i:
                dist.append(euclideanـdistance(p1, p2))
    euclideanـdistance_train.append(dist)    
print("Duration: %s seconds" % (time.time() - start_time)) 
   
start_time = time.time()
for im in test_features_list2:
    dist = []
    for i, p1 in enumerate(im):
        for j, p2 in enumerate(im):
            if j > i:
                dist.append(euclideanـdistance(p1, p2))
    euclideanـdistance_test.append(dist)    
print("Duration: %s seconds" % (time.time() - start_time)) 
      

Duration: 15.379674196243286 seconds
Duration: 3.8808131217956543 seconds


In [54]:
len(euclideanـdistance_train[0])

3003

In [55]:
start_time = time.time()
model = LDA()
model.fit(np.array(euclideanـdistance_train), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(euclideanـdistance_train), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(euclideanـdistance_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Train Duration : 4.265049934387207 seconds
Train Accuracy Durtion: 0.2284531593322754 seconds
Train Accuracy: 1.0
Test Accuracy Duration: 0.10071396827697754 seconds
Test Accuracy: 0.3532934131736527


In [64]:
### euclideanـistance and pca and lda 
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(euclideanـdistance_train))
model = LDA()
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(euclideanـdistance_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

Train Duration : 0.8664889335632324 seconds
Train Accuracy Durtion: 0.0017848014831542969 seconds
Train Accuracy: 0.688344172086043
Test Accuracy Duration: 0.07552528381347656 seconds
Test Accuracy: 0.5469061876247505


/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


In [72]:
import csv
with open('some.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(someiterable)

NameError: name 'someiterable' is not defined

In [ ]:
import csv

with open('PCA_euclidean_features.csv', 'w') as csvfile:
    fieldnames = ['*', 'last_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})

In [74]:
reduced_features.shape

(1999, 100)

In [77]:
import pandas as pd 
pd.DataFrame(reduced_features).to_csv("PCA_Euclidean_Features_Train.csv")
pd.DataFrame(reduced_test).to_csv("PCA_Euclidean_Features_Test.csv")
